In [2]:
from google.cloud import bigquery as bq
from google.cloud import bigquery_storage as bq_storage
import plotly.express as px
import pandas as pd

# create clients
client = bq.Client.from_service_account_json("key.json")
storage_client = bq_storage.BigQueryReadClient.from_service_account_json("key.json")

bq_dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")
bq_dataset = client.get_dataset(bq_dataset_ref)


In [2]:
#### GET react, angular, vue.js QUESTIONS ##############

sample_size = 100000

query = '''
    SELECT RAND() as r, tags, creation_date FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE REGEXP_CONTAINS(tags, r"vue") OR REGEXP_CONTAINS(tags, r"react") OR REGEXP_CONTAINS(tags, r"angular")
     OR REGEXP_CONTAINS(tags, r"svelte")  OR REGEXP_CONTAINS(tags, r"ember.js")  OR REGEXP_CONTAINS(tags, r"jquery")
    ORDER BY r
    LIMIT ''' + str(sample_size)


df = (
    client.query(query)
    .result()
    .to_dataframe(bqstorage_client=storage_client)
)

df

,r,tags,creation_date
0,4.698826e-07,javascript|angular|youtube-iframe-api,2018-09-10 08:22:30.697000+00:00
1,7.680278e-07,jquery,2012-09-03 19:13:07.620000+00:00
2,7.896342e-07,javascript|jquery|html|css,2014-05-15 02:48:34.943000+00:00
3,1.121077e-06,javascript|angularjs,2016-04-20 11:28:03.967000+00:00
4,1.306132e-06,jquery|jquery-selectors|checkbox,2013-02-05 11:41:33.790000+00:00
...,...,...,...
99995,5.169622e-02,jquery|ios|safari|webkit|responsive-design,2012-01-23 11:15:02.890000+00:00
99996,5.169702e-02,javascript|php|jquery|html,2016-11-22 19:52:53.617000+00:00
99997,5.169705e-02,javascript|jquery,2014-04-07 19:46:00.343000+00:00
99998,5.169764e-02,angular|angular7,2019-12-03 10:10:16.097000+00:00


In [3]:
##### FRONTEND 1 CHANGE TAG, SAVE TO CSV ################

def setTag(x):
    if(x.find('react') != -1):
        return 'react'
    elif(x.find('angular') != -1):
        return 'angular'
    elif(x.find('vue.js') != -1):
        return 'vue.js'
    elif(x.find('ember.js') != -1):
        return 'ember.js'
    elif(x.find('svelte') != -1):
        return 'svelte'
    elif(x.find('jquery') != -1):
        return 'jquery'
    

toSaveDf = pd.DataFrame(df)
toSaveDf = toSaveDf.drop(columns=['r'])
toSaveDf.rename(columns={'tags':'framework'}, inplace=True)
toSaveDf['framework'] = toSaveDf['framework'].map(setTag)
toSaveDf.to_csv('js-framework-tags-dataframe.csv')

In [10]:
## FRONTEND FRAMEWORK 2 ##
sample_size = 1000

query = '''
    SELECT RAND() as r, tags, creation_date FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE ( REGEXP_CONTAINS(tags, r"html") 
        OR REGEXP_CONTAINS(tags, r"javascript")
        OR REGEXP_CONTAINS(tags, r"HTML")
        OR REGEXP_CONTAINS(tags, r"css")
        OR REGEXP_CONTAINS(tags, r"CSS"))
    AND NOT REGEXP_CONTAINS(tags, r"vue")
    AND NOT REGEXP_CONTAINS(tags, r"angular")
    AND NOT REGEXP_CONTAINS(tags, r"react")
    AND NOT REGEXP_CONTAINS(tags, r"express")
    AND NOT REGEXP_CONTAINS(tags, r"ember")
    AND NOT REGEXP_CONTAINS(tags, r"node")
    ORDER BY r
    LIMIT ''' + str(sample_size)

query_new = '''
    SELECT RAND() as r, tags, creation_date FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE REGEXP_CONTAINS(tags, r"vue") OR REGEXP_CONTAINS(tags, r"react") OR REGEXP_CONTAINS(tags, r"angular")
    OR REGEXP_CONTAINS(tags, r"svelte")  OR REGEXP_CONTAINS(tags, r"ember")
    ORDER BY r
    LIMIT ''' + str(sample_size)


df_frontend2_old = (
    client.query(query)
    .result()
    .to_dataframe(bqstorage_client=storage_client)
)

df_frontend2_new = (
    client.query(query_new)
    .result()
    .to_dataframe(bqstorage_client=storage_client)
)

print(df_frontend2_old)
print(df_frontend2_new)

                r                                               tags  \
0    6.413625e-07                                javascript|momentjs   
1    1.477651e-06            html|css|css-transitions|css-transforms   
2    2.721226e-06             javascript|jquery|dom|jquery-selectors   
3    3.010372e-06  c#|javascript|jquery|asp.net|facebook-social-p...   
4    3.151565e-06                                  javascript|jquery   
..            ...                                                ...   
995  4.107521e-04                                           css|html   
996  4.107950e-04                             javascript|jquery|json   
997  4.111445e-04                       javascript|regex|regex-group   
998  4.117966e-04                    javascript|c++|recursion|vector   
999  4.124090e-04                            javascript|ios|html|url   

                       creation_date  
0   2019-04-16 05:11:55.350000+00:00  
1   2013-12-28 03:33:43.107000+00:00  
2   2011-09-06 10:

In [16]:
##### FRONTEND 2 CHANGE TAG, SAVE TO CSV ################

def setTag(x):
    if(
        x.find('react') != -1 or
        x.find('angular') != -1 or
        x.find('ember') != -1 or
        x.find('vue') != -1 or
        x.find('svelte') != -1
    ):
        return 'new'
    else:
        return 'old'
    
df_frontend2_new
df_frontend2_old
toSaveDf = df_frontend2_new.append(df_frontend2_old)
toSaveDf = toSaveDf.drop(columns=['r'])
toSaveDf.rename(columns={'tags':'technology'}, inplace=True)
toSaveDf['technology'] = toSaveDf['technology'].map(setTag)
toSaveDf.to_csv('old_vs_new_frontend.csv')